In [1]:
import praw
import psaw
import numpy
%matplotlib inline
from pylab import *
from pprint import pprint
import pandas as pd
import simplejson as json
from pmaw import PushshiftAPI
import pyspark
import datetime as dt

%config InlineBackend.figure_format='retina'

In [40]:
def aggregate(df): #aggregate over the subreddits so that we get a list with subreddit - crosspost from and counts
    
    df2 = df.groupby(['subreddit','crosspost_parent']).agg({'subreddit_subscribers': 'mean', 'crosspost_parent_subs': 'mean' , 'author' : 'count', 'crosspost_parent_num': 'sum'}).reset_index()
    df2.rename(columns = {'author':'count',}, inplace = True)
    df2['crosspost_parent_subs'] = df2['crosspost_parent_subs'].fillna(0)
    df2['crosspost_parent_subs'] = df2['crosspost_parent_subs'].astype(int)

    imp = df2.groupby(['crosspost_parent']).agg({'subreddit': 'count', 'count': 'sum'})
    imp = imp.rename(columns ={'count': 'total'}) 
    imp.drop(['subreddit'], axis=1, inplace=True)
    df3 = df2.merge(imp, on='crosspost_parent', how='left')

    print('aggregated')
    return df3

In [41]:
def get_crosspost_child(df, outfile, depth_lim): #find the crosspost children of posts from the observed subreddits
    # api = PushshiftAPI()
    # df = df[df['num_crossposts'] > 0] # We do this by looking for posts that have been crossposted
    # t = df.groupby(['url']).agg({'num_crossposts': 'sum', 'id':  'max'}).reset_index()
    # urls = list(t['url']) # then collect the url of posts / attached links / pictures

    # df = pd.DataFrame(columns=['id', 'url', 'title', 'subreddit', 'selftext', 'subreddit_subscribers',
    #     'num_crossposts', 'crosspost_parent', 'created_utc', 'author', 'num_comments', 'score'])

    # for j in urls:
    #     results2 = api.search_submissions(
    #         url = j, # and search for them using the pushshift api
    #         filter=[ 'id', 'url', 'title', 'subreddit', 'selftext', 'subreddit_subscribers',
    #         'num_crossposts', 'crosspost_parent', 'created_utc', 'author', 'num_comments', 'score']) 
    #     temp = pd.DataFrame([thing for thing in results2])
    #     df = pd.concat([df, temp])

    # df.to_pickle(f'../../Files/{outfile}_raw_child_{depth_lim}.pickle')
    df2 = df[df['num_crossposts'] > 0].reset_index(drop=True) #split into parent posts (number of crossposts > 0 )
    df3 = df[df['num_crossposts'] == 0].reset_index(drop=True) # and child posts (number of crossposts == 0)
    df3 = df3[df3['crosspost_parent'].notna()] # Drop all children where field for parent is empty
    df3['crosspost_parent']  = df3['crosspost_parent'].apply(lambda x: x[3:])
    df2.drop('crosspost_parent', axis=1, inplace=True)
    df2 = df2.rename(columns ={'subreddit':'crosspost_parent', 'subreddit_subscribers': 'crosspost_parent_subs', 'num_crossposts': 'crosspost_parent_num'})
    df4 = df2.merge(df3, left_on='id', right_on='crosspost_parent', suffixes= ('','_y'))

    df4 = df4[(df4['crosspost_parent'].str.startswith('u_') == False) & (df4['subreddit'].str.startswith('u_') == False)] # get rid off users as subreddit
    df5 = aggregate(df4)
    
    df
    print(f'identified {len(df4)} children')
    df5.to_pickle(f'../../Files/{outfile}_cross_child_{depth_lim}.pickle') # save file to avoid straining the API
    return df5

In [4]:
df = pd.read_pickle('/Users/jakobschlierf/Desktop/Master/Thesis/Files/test_0613_raw_child_2.pickle')


In [42]:
df.head()

,id,url,title,subreddit,selftext,subreddit_subscribers,num_crossposts,crosspost_parent,created_utc,author,num_comments,score
0,n6yugl,https://pdf.math.dev/,Beautiful PDFs from HTML,u_dweeed,,0,0,t3_n6yjsj,1620395121,dweeed,0,1
1,n6yjsj,https://pdf.math.dev/,Beautiful PDFs from HTML,webdev,,821591,0,NaN,1620394253,speckz,19,1
2,n6yjm4,https://pdf.math.dev/,Beautiful PDFs from HTML,web_design,,536815,0,NaN,1620394240,speckz,6,1
3,n6yjju,https://pdf.math.dev/,Beautiful PDFs from HTML,Frontend,,135880,0,NaN,1620394235,speckz,0,1
4,mli1c7,https://pdf.math.dev,Guide (and example code) to producing beautifu...,css,,82004,0,NaN,1617733009,RentGreat8009,2,1


In [43]:
df2 = get_crosspost_child(df, 'serverfix', 2 )

aggregated
identified 15950 children


In [44]:
df2.head()

,subreddit,crosspost_parent,subreddit_subscribers,crosspost_parent_subs,count,crosspost_parent_num,total
0,0sanitymemes,interestingasfuck,9076.0,7595870,1,16,541
1,111bookmarks,ATBGE,1.0,1418326,1,2,481
2,1200isfineIGUESSugh,peopleofwalmart,50073.0,409263,1,1,13
3,13or30,HumansBeingBros,406092.0,2215179,1,7,144
4,13or30,sadcringe,404388.0,924672,1,4,63


In [45]:
df3 = pd.read_pickle('../../Files/test_0613_cross_parent_2.pickle')

In [46]:
df3.head()

,subreddit,crosspost_parent,subreddit_subscribers,crosspost_parent_subs,count,crosspost_parent_num,total
0,6thForm,2meirl4meirl,35791.0,1268402,1,3,111
1,6thForm,6thForm,37984.0,37984,4,8,4
2,6thForm,Advice,37341.0,512881,1,1,11
3,6thForm,AlevelGeog,41873.0,159,1,1,1
4,6thForm,ApplyingToCollege,38328.5,325892,2,2,6


In [48]:
df4 = pd.concat([df3, df2])

In [51]:
df2.shape

(8863, 7)

In [49]:
df4.shape

(30629, 7)

In [52]:
df4.to_pickle('../../Files/test_0613_cross.pickle')

In [78]:
CLIENT_ID = '5FpjYNUrv93rDqflbNE-6w'
CLIENT_SECRET = '5ZITWnmNd6_qCbPXDB90xmOaIMo9_w'
USER_NAME = 'theonejay96'
SCRIPT_NAME = 'Data\ Science\ Thesis'
LOGIN_PASSWORD = 'vejkyz-hyBfez-4cinte'

reddit =  praw.Reddit(
    client_id= CLIENT_ID ,
    client_secret= CLIENT_SECRET ,
    user_agent= SCRIPT_NAME,
    username= USER_NAME,
    password= LOGIN_PASSWORD)

api = PushshiftAPI()

In [37]:
api = PushshiftAPI()

In [82]:
gen2 = api.search_submissions(
    filter=[ 'id', 'url', 'title', 'subreddit', 'subreddit_id', 'subreddit_subscribers',
        'num_crossposts', 'crosspost_parent', 'crosspost_parent_list', 'created_utc', 'author'],
    limit= 100)

temp = pd.DataFrame([thing for thing in gen2])
temp.head()

Not all PushShift shards are active. Query results may be incomplete.


,author,created_utc,id,num_crossposts,subreddit,subreddit_id,subreddit_subscribers,title,url,crosspost_parent,crosspost_parent_list
0,David_McTravord555,1654625979,v73j9f,0,worldbuilding,t5_2rd6n,905089,In my universe where magical creatures (like v...,https://www.reddit.com/r/worldbuilding/comment...,NaN,NaN
1,Antdryem_,1654625979,v73j9d,0,Dublin,t5_2r1hz,76494,Getting to live in Dublin,https://www.reddit.com/r/Dublin/comments/v73j9...,NaN,NaN
2,Spnkmeimbd,1654625979,v73j9a,0,sideboobandass,t5_2pam8b,3271,Come keep me warm,https://i.redd.it/y7heujw9uy391.jpg,t3_v5yhln,"[{'all_awardings': [], 'allow_live_comments': ..."
3,johnevepierrot,1654625979,v73j99,0,Bromance2,t5_3h6m2l,383,Looking for Connections,https://www.reddit.com/r/Bromance2/comments/v7...,NaN,NaN
4,pirl_opden,1654625979,v73j97,0,HiTMAN,t5_2rn0x,134437,"Another JIRA ticket,another set of funerals",https://v.redd.it/w7e46i2bo8491,NaN,NaN


In [80]:
g = [ting for ting in gen2]

In [79]:

temp = pd.DataFrame([thing for thing in gen2])
temp.head()

""


In [81]:
g

[]

In [71]:
l = t.crosspost_parent_list.tolist()

In [72]:
for i in l:
    if len(i) > 0:
        print(i)

TypeError: object of type 'float' has no len()

In [76]:
l[6][0]['subreddit']

'entertainment'

In [207]:
df = pd.read_pickle('/Users/jakobschlierf/Desktop/Master/Thesis/Files/test0606__cross.pickle')

In [208]:
df.head()

,subreddit,subreddit_id,crosspost_parent,crosspost_parent_id,subreddit_subscribers,crosspost_parent_subs,count,crosspost_parent_num,total
0,CovidVaccinated,t5_3gppqs,MapPorn,t5_2si92,4351.0,1252559,1,1,1
1,CovidVaccinated,t5_3gppqs,dataisbeautiful,t5_2tk95,4196.0,15347693,1,2,1
2,DebateVaccines,t5_3jwf7,AntiVaxxers,t5_384mz,2354.5,27397,2,26,2
3,DebateVaccines,t5_3jwf7,ConspiracyUltra,t5_3flq3y,2711.869565,236,23,23,23
4,DebateVaccines,t5_3jwf7,Coronavirus,t5_2x4yx,2585.5,2365057,4,5,12


In [4]:
df = pd.read_pickle('/Users/jakobschlierf/Desktop/Master/Thesis/Files/test0606__ratio_temp2')
df.head()

,subreddit,crosspost_parent,crosspost_child,total,crosspost_parent_%,crosspost_child_%
0,ABoringDystopia,857,10,2393,0.358128,0.004179
1,ATBGE,14,408,2400,0.005833,0.170000
2,ActualPublicFreakouts,823,49,2382,0.345508,0.020571
3,AgainstHateSubreddits,1,7,1908,0.000524,0.003669
4,AlternativeHealth,42,0,245,0.171429,0.000000


In [19]:
df = pd.read_pickle('/Users/jakobschlierf/Desktop/Master/Thesis/Files/test0606__cross_child.pickle')
df.head()

,subreddit,subreddit_id,crosspost_parent,crosspost_parent_id,subreddit_subscribers,crosspost_parent_subs,count,crosspost_parent_num,total
0,13or30,t5_3mzz4,Conservative,t5_2qh6p,406093.0,717882,1,2,89
1,1950s,t5_2suyd,interestingasfuck,t5_2qhsa,6798.0,7474756,1,3,781
2,197,t5_3enhu,DankLeft,t5_11ga7z,16225.0,157790,5,5,63
3,2007scape,t5_2wbww,ThatsInsane,t5_swxxz,557307.0,862697,1,22,164
4,2007scape,t5_2wbww,funny,t5_2qh33,550208.0,35286028,1,11,146


In [25]:
df.shape

(4881, 9)

In [22]:
# df4 = df4[(df4['crosspost_parent'].str.contains('u_') == False)]
# df2 = df[(df['crosspost_parent'].str.startswith('u_') == False)]

df2 = df[(df['crosspost_parent'].str.startswith('u_') == False) & (df['subreddit'].str.startswith('u_') == False)]

In [24]:
df2.shape

(4102, 9)

In [197]:
df2 = df[df['crosspost_parent_list'].notna()].reset_index()
df2 = df2[df2['crosspost_parent_list'].str.len() != 0] #sometimes, this field contains an empty list

df2['t'] = df2['crosspost_parent_list'].apply(lambda x: dict(x[0])) # Pull crosspost_from information from field 'crosspost_parent_list' (which is in a json format)
df2['crosspost_parent'] = df2['t'].apply(lambda x: x['subreddit'])  # Using the field has the advantage that we can deal with deleted posts which we could not find using the reddit API.
df2['crosspost_parent_id'] = df2['t'].apply(lambda x: x['subreddit_id'])
df2['crosspost_parent_subs'] = df2['t'].apply(lambda x: x['subreddit_subscribers'] )
df2['crosspost_parent_num'] = df2['t'].apply(lambda x: x['num_crossposts'] )


In [198]:
df2.head()

,index,id,url,title,subreddit,subreddit_id,subreddit_subscribers,crosspost_parent,crosspost_parent_list,num_crossposts,created_utc,author,created,t,crosspost_parent_id,crosspost_parent_subs,crosspost_parent_num
0,3,l9ncqc,https://i.redd.it/i214kiu1jqe61.jpg,What’s better than freezing to death,ABoringDystopia,t5_3c6l1,514355,WhitePeopleTwitter,"[{'all_awardings': [], 'allow_live_comments': ...",0,1612130867,doctornowzaradan,1.612124e+09,"{'all_awardings': [], 'allow_live_comments': F...",t5_35n7t,2011831,1
1,5,l9m64s,https://v.redd.it/28r4g0xudne61,How are those $2000 relief checks coming along?,ABoringDystopia,t5_3c6l1,514354,NewDealAmerica,[{'all_awardings': [{'award_sub_type': 'GLOBAL...,0,1612127619,ProctorZeuss,1.612120e+09,{'all_awardings': [{'award_sub_type': 'GLOBAL'...,t5_2xl4f0,43387,6
2,6,l9kxsi,https://i.redd.it/d0u3bosh3me61.jpg,Jimmy is a fucking badass,ABoringDystopia,t5_3c6l1,514347,nextfuckinglevel,[{'all_awardings': [{'award_sub_type': 'PREMIU...,0,1612124273,[deleted],1.612117e+09,{'all_awardings': [{'award_sub_type': 'PREMIUM...,t5_m0bnr,3622511,25
3,7,l9kltk,https://i.redd.it/kbybo0r1ppe61.jpg,School staff is one of the most important peop...,ABoringDystopia,t5_3c6l1,514347,MadeMeSmile,[{'all_awardings': [{'award_sub_type': 'GLOBAL...,0,1612123409,AnoonymouseChocobo,1.612116e+09,{'all_awardings': [{'award_sub_type': 'GLOBAL'...,t5_2uqcm,2852714,3
4,9,l9kepr,https://i.redd.it/k7scw0fiqpe61.jpg,Wtf children are not fined for late loans in m...,ABoringDystopia,t5_3c6l1,514346,wholesomememes,[{'all_awardings': [{'award_sub_type': 'GLOBAL...,0,1612122905,Damroth,1.612116e+09,{'all_awardings': [{'award_sub_type': 'GLOBAL'...,t5_3gcwj,8871710,1


In [206]:
df3 = df2[(df2['crosspost_parent'].str.contains('u_') == False)]
df3

,index,id,url,title,subreddit,subreddit_id,subreddit_subscribers,crosspost_parent,crosspost_parent_list,num_crossposts,created_utc,author,created,t,crosspost_parent_id,crosspost_parent_subs,crosspost_parent_num
0,3,l9ncqc,https://i.redd.it/i214kiu1jqe61.jpg,What’s better than freezing to death,ABoringDystopia,t5_3c6l1,514355,WhitePeopleTwitter,"[{'all_awardings': [], 'allow_live_comments': ...",0,1612130867,doctornowzaradan,1.612124e+09,"{'all_awardings': [], 'allow_live_comments': F...",t5_35n7t,2011831,1
1,5,l9m64s,https://v.redd.it/28r4g0xudne61,How are those $2000 relief checks coming along?,ABoringDystopia,t5_3c6l1,514354,NewDealAmerica,[{'all_awardings': [{'award_sub_type': 'GLOBAL...,0,1612127619,ProctorZeuss,1.612120e+09,{'all_awardings': [{'award_sub_type': 'GLOBAL'...,t5_2xl4f0,43387,6
2,6,l9kxsi,https://i.redd.it/d0u3bosh3me61.jpg,Jimmy is a fucking badass,ABoringDystopia,t5_3c6l1,514347,nextfuckinglevel,[{'all_awardings': [{'award_sub_type': 'PREMIU...,0,1612124273,[deleted],1.612117e+09,{'all_awardings': [{'award_sub_type': 'PREMIUM...,t5_m0bnr,3622511,25
3,7,l9kltk,https://i.redd.it/kbybo0r1ppe61.jpg,School staff is one of the most important peop...,ABoringDystopia,t5_3c6l1,514347,MadeMeSmile,[{'all_awardings': [{'award_sub_type': 'GLOBAL...,0,1612123409,AnoonymouseChocobo,1.612116e+09,{'all_awardings': [{'award_sub_type': 'GLOBAL'...,t5_2uqcm,2852714,3
4,9,l9kepr,https://i.redd.it/k7scw0fiqpe61.jpg,Wtf children are not fined for late loans in m...,ABoringDystopia,t5_3c6l1,514346,wholesomememes,[{'all_awardings': [{'award_sub_type': 'GLOBAL...,0,1612122905,Damroth,1.612116e+09,{'all_awardings': [{'award_sub_type': 'GLOBAL'...,t5_3gcwj,8871710,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8787,95,ltoqoj,https://i.redd.it/tb78uxank0k61.jpg,Please join us #Condemns_the_terrorist_activit...,UnitedNations,t5_2s0h9,6426,myanmar,"[{'all_awardings': [], 'allow_live_comments': ...",0,1614434546,memelgi,1.614427e+09,"{'all_awardings': [], 'allow_live_comments': F...",t5_2rt95,8938,1
8788,96,lto7a1,https://v.redd.it/scqr8x6vyyj61,It's no longer safe for us. She is just a frui...,UnitedNations,t5_2s0h9,6425,myanmar,"[{'all_awardings': [], 'allow_live_comments': ...",0,1614432753,memelgi,1.614426e+09,"{'all_awardings': [], 'allow_live_comments': F...",t5_2rt95,8937,1
8789,97,lto68o,https://i.redd.it/wi6qozresyj61.jpg,A pregnant woman was arrested by myanmar brutu...,UnitedNations,t5_2s0h9,6425,myanmar,"[{'all_awardings': [], 'allow_live_comments': ...",0,1614432653,memelgi,1.614425e+09,"{'all_awardings': [], 'allow_live_comments': F...",t5_2rt95,8937,1
8790,98,m5cvc5,/r/LibertarianPartyUSA/comments/m5cpqj/liberta...,Libertarian startup city Próspera has broken g...,GoldandBlack,t5_3ffci,79304,LibertarianPartyUSA,"[{'all_awardings': [], 'allow_live_comments': ...",0,1615783183,nixfu,1.615776e+09,"{'all_awardings': [], 'allow_live_comments': F...",t5_3epv4,7285,1


In [200]:
df2 = df2[df2['crosspost_parent'] != 'u_']

In [193]:
len(row['crosspost_parent_list'][0]['subreddit'])

18

In [195]:
res = []
for i, row in df.iterrows():
    if type(row['crosspost_parent_list']) == list:
        if len(row['crosspost_parent_list']) >= 3:
            if row['crosspost_parent_list'][0]['subreddit'][0:2] == 'u_':
                res.append(row['crosspost_parent_list'])

res

[]

In [150]:
t = df.iloc[3]['crosspost_parent_list']

In [155]:
t[0]['subreddit'][0:2]

'Wh'

In [143]:
df['t'] = df['crosspost_parent_list'].apply(lambda x: dict(x[0]))


TypeError: 'float' object is not subscriptable

In [132]:

df23 = df[df['crosspost_parent'].notna()]
# df23 = df23[df23['crosspost_parent'].apply(lambda x: x[0:1]) != 't3']
# df23.head()

In [126]:
df23.shape

(8792, 12)

In [82]:
df4 = df[df['crosspost_parent'].apply(lambda x: x[0:2]) != 'u_']
df4.shape

TypeError: 'float' object is not subscriptable

In [71]:

df3 = df2.groupby('subreddit').agg({'crosspost_parent': 'count', 'num_crossposts': lambda x: np.count_nonzero(x), 'subreddit_subscribers' : 'count'}).reset_index()
df3 = df3.rename(columns ={'subreddit_subscribers':'total', 'num_crossposts': 'crosspost_child'})

In [72]:
df3.sort_values('crosspost_parent',ascending=False)

,subreddit,crosspost_parent,crosspost_child,total
135,u_rklokh,900,0,900
9,Astuff,503,9,755
103,covidskepticscanada,347,15,603
0,ABoringDystopia,325,9,900
57,NEWPOLITIC,311,22,900
...,...,...,...,...
115,montreal,0,3,897
122,privacy,0,13,891
116,news,0,19,900
120,pcgaming,0,4,900


In [73]:
df3['crosspost_parent_%'] = df3['crosspost_parent'] / df3['total']
df3['crosspost_child_%'] = df3['crosspost_child'] / df3['total']

In [78]:
df3['crosspost_parent_%'].mean()

0.20008680636389625

In [79]:
df3['crosspost_child_%'].mean()

0.019490443440403512

In [255]:
df = pd.DataFrame(columns=['id', 'url', 'title','subreddit', 'subreddit_id', 'subreddit_subscribers',
        'num_crossposts', 'crosspost_parent', 'created_utc', 'author'])
for i in urls:
    gen2 = api.search_submissions(
    url = i ,
    filter=[ 'id', 'url', 'title', 'subreddit', 'subreddit_id', 'subreddit_subscribers',
        'num_crossposts', 'crosspost_parent', 'created_utc', 'author'],
    limit = 100) #limit to avoid going over the rate limit.
    results2 = list(gen2)
    temp = pd.DataFrame([thing.d_ for thing in results2])
    df = pd.concat([df, temp])

df2 = df[df['num_crossposts'] > 0].reset_index(drop=True)
df3 = df[df['num_crossposts'] == 0].reset_index(drop=True)
df3 = df3[df3['crosspost_parent'].notna()]
df3['crosspost_parent']  = df3['crosspost_parent'].apply(lambda x: x[3:])
df2.drop('crosspost_parent', axis=1, inplace=True)
df2 = df2.rename(columns ={'subreddit':'crosspost_parent','subreddit_id': 'crosspost_parent_id', 'subreddit_subscribers': 'crosspost_parent_subs', 'num_crossposts': 'crosspost_parent_num'})
df4 = df2.merge(df3, left_on='id', right_on='crosspost_parent', suffixes= ('','_y'))
df5 = df4.groupby(['subreddit','subreddit_id','crosspost_parent', 'crosspost_parent_id']).agg({'subreddit_subscribers': 'mean', 'crosspost_parent_subs': 'mean' , 'author' : 'count', 'crosspost_parent_num': 'sum'}).reset_index()
df5.rename(columns = {'author':'count',}, inplace = True)
df5['crosspost_parent_subs'] = df5['crosspost_parent_subs'].astype(int)


/Users/jakobschlierf/opt/anaconda3/envs/reddit_env_test/lib/python3.9/site-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
/Users/jakobschlierf/opt/anaconda3/envs/reddit_env_test/lib/python3.9/site-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
/Users/jakobschlierf/opt/anaconda3/envs/reddit_env_test/lib/python3.9/site-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
/Users/jakobschlierf/opt/anaconda3/envs/reddit_env_test/lib/python3.9/site-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


KeyboardInterrupt: 

In [3]:
# df4.to_pickle('../../Files/merge_test.pickle')
df4 = pd.read_pickle('../../Files/test_child.pickle')

In [6]:
df4.head()
df = df4

In [7]:
df2 = df[df['num_crossposts'] > 0].reset_index(drop=True)
df3 = df[df['num_crossposts'] == 0].reset_index(drop=True)
df3 = df3[df3['crosspost_parent'].notna()]
df3['crosspost_parent']  = df3['crosspost_parent'].apply(lambda x: x[3:])
df2.drop('crosspost_parent', axis=1, inplace=True)
df2 = df2.rename(columns ={'subreddit':'crosspost_parent','subreddit_id': 'crosspost_parent_id', 'subreddit_subscribers': 'crosspost_parent_subs', 'num_crossposts': 'crosspost_parent_num'})
df4 = df2.merge(df3, left_on='id', right_on='crosspost_parent', suffixes= ('','_y'))
df5 = df4.groupby(['subreddit','subreddit_id','crosspost_parent', 'crosspost_parent_id']).agg({'subreddit_subscribers': 'mean', 'crosspost_parent_subs': 'mean' , 'author' : 'count', 'crosspost_parent_num': 'sum'}).reset_index()
df5.rename(columns = {'author':'count',}, inplace = True)
df5['crosspost_parent_subs'] = df5['crosspost_parent_subs'].astype(int)

In [315]:
df5 = df2.merge(df3, left_on='id', right_on='crosspost_parent', suffixes= ('','_y'))


In [324]:
df5 = df4.groupby(['subreddit','subreddit_id','crosspost_parent', 'crosspost_parent_id']).agg({'subreddit_subscribers': 'mean', 'crosspost_parents_subs': 'mean' , 'author' : 'count', 'crosspost_parent_num': 'sum'}).reset_index()
# df3.rename(columns = {'author':'count',}, inplace = True)
# df3['crosspost_from_subs'] = df3['crosspost_from_subs'].astype(int)
    

In [47]:
k = df5.columns
for j in k:
    print(j)

subreddit
subreddit_id
crosspost_parent
crosspost_parent_id
subreddit_subscribers
crosspost_parent_subs
count
crosspost_parent_num


In [14]:
s = df5['subreddit'].drop_duplicates()
s.append(df5['crosspost_parent'].drop_duplicates())
s

/var/folders/fh/yf9jwszj6d5_1_xhfz_l6k000000gn/T/ipykernel_28294/726648991.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  s.append(df5['crosspost_parent'].drop_duplicates())


0         ABoringDystopia
2            AconspiracyA
3       AlternativeHealth
5      Anarcho_Capitalism
7        AnythingGoesNews
              ...        
163          vacci_nation
164           vandwellers
165              walkaway
166     whatsreallygoinon
167             worldnews
Name: subreddit, Length: 126, dtype: object

In [17]:

for i in s:
    print(i)

ABoringDystopia
AconspiracyA
AlternativeHealth
Anarcho_Capitalism
AnythingGoesNews
Astuff
BidenLies
COVID
CeticosRespostaCovid
ChurchOfCOVID
CoVaxSkeptical
ConservativeMemes
ConspiracyUltra
CoronaKritiker
CoronaNC
CoronavirusCirclejerk
CoronavirusColorado
CoronavirusOntario
CoronavirusOregon
CoronavirusToronto
CoronavirusUK
CoronavirusUS
CoronavirusVariants
CovidIsACult
CovidVaccinated
DankLeft
DebateVaccine
DebateVaccines
DescentIntoTyranny
DrFauci
EndTheLockdowns
EnoughAntifaSpam
FUCKCHINAVIRUS
FuckFuckMasks
GreatReject
HillaryForPrison
JustUnsubbed
LeopardsAteMyFace
LibertarianLGBTQ
LockdownCriticalLeft
LockdownSkepticismCAN
LoveForDronePilots
Mujico
NEWPOLITIC
NoNewNormal
NoNewNormalBan
NoNoNewNormal
NoShitSherlock
NolibsWatch
OpenTheSchools
PeoplesPartyofCanada
Republican
RepublicanValues
RuralNewsNetwork
SatanicRitualAbuse
Slovakia
SurprisingNoOne
Thailand
TheBeliefInstinct
TheFightThatMatters
TheGreatResistance
ThePeoplesPower
TopConspiracy
TrueAntiVaccination
TruthLeaks
Unbiase

In [51]:
import numpy as np
np.savetxt("../../Files/depth1.csv", 
           s,
           delimiter =", ", 
           fmt ='% s')

In [35]:
t = df5['subreddit'].drop_duplicates().to_list()

In [39]:
t = df5['subreddit'].drop_duplicates().to_list()
t.extend(df5['crosspost_parent'].drop_duplicates().to_list())
res = []
for i in t:
    if i not in res and i not in subreddits:
        res.append(i)

AttributeError: 'list' object has no attribute 'drop_duplicates'

In [41]:

subreddits = ['DebateVaccines', 'CovidVaccinated', 'Vaccine', 'Coronavirus', 'LockdownSkepticism', 'HermanCainAward', 'NoNewNormal']
res = []
for i in t:
    if i not in res and i not in subreddits:
        res.append(i)

In [44]:
print(len(t), len(res))

177 129


In [45]:
res

['ABoringDystopia',
 'AconspiracyA',
 'AlternativeHealth',
 'Anarcho_Capitalism',
 'AnythingGoesNews',
 'Astuff',
 'BidenLies',
 'COVID',
 'COVID19_support',
 'CeticosRespostaCovid',
 'ChurchOfCOVID',
 'CoVaxSkeptical',
 'ConservativeMemes',
 'ConspiracyUltra',
 'CoronaKritiker',
 'CoronaNC',
 'CoronavirusCirclejerk',
 'CoronavirusColorado',
 'CoronavirusOntario',
 'CoronavirusOregon',
 'CoronavirusToronto',
 'CoronavirusUK',
 'CoronavirusUS',
 'CoronavirusVariants',
 'CovidIsACult',
 'DankLeft',
 'DebateVaccine',
 'DescentIntoTyranny',
 'DrFauci',
 'EndTheLockdowns',
 'EnoughAntifaSpam',
 'FUCKCHINAVIRUS',
 'FuckFuckMasks',
 'GoldandBlack',
 'GreatReject',
 'Health',
 'HillaryForPrison',
 'JustUnsubbed',
 'LeopardsAteMyFace',
 'Liberate_Canada',
 'LibertarianLGBTQ',
 'LockdownCriticalLeft',
 'LockdownSkepticismCAN',
 'LoveForDronePilots',
 'Mujico',
 'NEWPOLITIC',
 'NoNewNormalBan',
 'NoNoNewNormal',
 'NoShitSherlock',
 'NolibsWatch',
 'OpenTheSchools',
 'PeoplesPartyofCanada',
 'Repu

In [2]:
def importance(df): #calculate the importance (similar to tf-idf)
    # OPEN
    # - make work with Crosspost from
    imp = df.groupby(['crosspost_from']).agg({'subreddit': 'count', 'count': 'sum', 'crosspost_from_num' : 'sum'})
    imp = imp.rename(columns ={'count': 'total'}) 
    imp.drop(['subreddit'], axis=1, inplace=True)
    df2 = df.join(imp, on='crosspost_from', how='left')
    return df2

In [75]:
df2 = pd.read_pickle('../../Files/2021-01-02.pickle')

In [68]:
df3 = df2[df2['crosspost_parent_list'].str.len() == 0]

In [76]:
df2.head()

,index,subreddit,subreddit_id,subreddit_subscribers,crosspost_parent,crosspost_parent_list,created_utc,author,created,t,crosspost_from,crosspost_from_id,crosspost_from_subs,crosspost_from_num
0,0,DebateVaccines,t5_3jwf7,2347,t3_l9iiqp,"[{'all_awardings': [], 'allow_live_comments': ...",1612117931,[deleted],1.612111e+09,"{'all_awardings': [], 'allow_live_comments': F...",conspiracy,t5_2qh4r,1455036,1
1,3,DebateVaccines,t5_3jwf7,2325,t3_l81oft,[{'all_awardings': [{'award_sub_type': 'GLOBAL...,1612056522,doingbasiclifeprep,1.612049e+09,{'all_awardings': [{'award_sub_type': 'GLOBAL'...,IAmA,t5_2qzb6,20780922,1
2,15,DebateVaccines,t5_3jwf7,2317,t3_l7apag,[{'all_awardings': [{'award_sub_type': 'GLOBAL...,1611915334,EuCleo,1.611908e+09,{'all_awardings': [{'award_sub_type': 'GLOBAL'...,Coronavirus,t5_2x4yx,2363477,1
3,17,DebateVaccines,t5_3jwf7,2314,t3_l77ln3,"[{'all_awardings': [], 'allow_live_comments': ...",1611865305,dannylenwinn,1.611858e+09,"{'all_awardings': [], 'allow_live_comments': F...",ForUnitedStates,t5_2i3wxn,2757,26
4,18,DebateVaccines,t5_3jwf7,2314,t3_l746cr,"[{'all_awardings': [], 'allow_live_comments': ...",1611857406,dannylenwinn,1.611850e+09,"{'all_awardings': [], 'allow_live_comments': F...",ForUnitedStates,t5_2i3wxn,2754,17


In [ ]:
submission = reddit.submission('t3_p8ibgd')
submission.title

In [40]:
r['crosspost_parent_list'][0]

IndexError: list index out of range

In [26]:
df2 = df2[df2['crosspost_parent'].notna()].reset_index()

In [52]:

df2.to_pickle(f'../../Files/test.pickle')

IndexError: list index out of range

# PySpark Test

In [6]:
from pyspark import SparkConf, SparkContext 
sc = SparkContext("local[4]", 'spark_test')





ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=spark_test, master=local[4]) created by __init__ at /tmp/ipykernel_320226/2751826978.py:2 

In [36]:
lines = sc.textFile("Files/RS_2005-07.bz2", )

In [42]:
# lines.map(lambda x: x[0]).take(1)
lines.take(1)

['{"archived":true,"author":"zaz","author_flair_background_color":null,"author_flair_css_class":null,"author_flair_richtext":[],"author_flair_text":null,"author_flair_text_color":null,"author_flair_type":"text","brand_safe":true,"can_gild":true,"contest_mode":false,"created_utc":1120181161,"distinguished":null,"domain":"cnn.com","edited":false,"gilded":0,"hidden":false,"hide_score":false,"id":"204","is_crosspostable":true,"is_reddit_media_domain":false,"is_self":false,"is_video":false,"link_flair_css_class":null,"link_flair_richtext":[],"link_flair_text":null,"link_flair_text_color":"dark","link_flair_type":"text","locked":false,"media":null,"media_embed":{},"no_follow":false,"num_comments":0,"num_crossposts":0,"over_18":false,"parent_whitelist_status":"all_ads","permalink":"\\/r\\/reddit.com\\/comments\\/204\\/cnncom_dj_admits_false_tale_about_missing_teen\\/","rte_mode":"markdown","score":4,"secure_media":null,"secure_media_embed":{},"selftext":"","send_replies":true,"spoiler":false,